In [1]:
!pip install tensorflow-recommenders
!pip install pprintpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.4 MB/s eta 0:00:00


In [6]:
%cd \Users\darth\PycharmProjects\FlavorFinder\flavorfinder

C:\Users\darth\PycharmProjects\FlavorFinder\flavorfinder


C:\Users\darth\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

interaction_data = pd.read_csv("data/RAW_interactions.csv")
recipe_data = pd.read_csv("data/RAW_recipes.csv")

interaction_train = pd.read_csv("data/interactions_train.csv")
interaction_test = pd.read_csv("data/interactions_test.csv")
interaction_data = interaction_data.astype({'user_id': 'string', 'recipe_id':'string'})
interaction_train = interaction_train.astype({'user_id': 'string', 'recipe_id':'string'})
interaction_test = interaction_test.astype({'user_id': 'string', 'recipe_id':'string'})
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        self.user_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=uniqueUserIds, mask_token=None),
                                        # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(uniqueUserIds)+1, embedding_dimension)
                                    ])

        self.product_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=uniqueFoodIds, mask_token=None),
                                    # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(uniqueFoodIds)+1, embedding_dimension)
                                    ])
        # Set up a retrieval task and evaluation metrics over the
        # entire dataset of candidates.
        self.ratings = tf.keras.Sequential([
                            tf.keras.layers.Dense(256, activation="relu"),
                            tf.keras.layers.Dense(64,  activation="relu"),
                            tf.keras.layers.Dense(1)
                              ])

    def call(self, userId, foodId):
        user_embeddings  = self.user_embeddings (userId)
        food_embeddings = self.product_embeddings(foodId)
        return self.ratings(tf.concat([user_embeddings, food_embeddings], axis=1))

# Build a model.
class FoodModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer   = tfrs.tasks.Ranking(
                                                    loss    =  tf.keras.losses.MeanSquaredError(),
                                                    metrics = [tf.keras.metrics.RootMeanSquaredError()])


    def compute_loss(self, features, training=False):
        rating_predictions = self.ranking_model(features["userID"], features["foodID"]  )

        return self.task( labels=features["rating"], predictions=rating_predictions)
uniqueUserIds = interaction_data.user_id.unique()
uniqueFoodIds = interaction_data.recipe_id.unique()
train_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(interaction_train.user_id.values, tf.string),
    "foodID":tf.cast(interaction_train.recipe_id.values, tf.string),
    "rating":tf.cast(interaction_train.rating.values, tf.float32)
})

test_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(interaction_test.user_id.values, tf.string),
    "foodID":tf.cast(interaction_test.recipe_id.values, tf.string),
    "rating":tf.cast(interaction_test.rating.values, tf.float32)
})
tf.random.set_seed(42)

train_data = train_data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
model = FoodModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001))
cached_train = train_data.shuffle(100_000).batch(8192).cache()
cached_test = test_data.batch(4096).cache()
model.fit(cached_train, epochs=10)

Epoch 1/10
86/86 [==============================] - 5s 42ms/step - root_mean_squared_error: 4.4918 - loss: 20.1493 - regularization_loss: 0.0000e+00 - total_loss: 20.1493
Epoch 2/10
86/86 [==============================] - 1s 10ms/step - root_mean_squared_error: 4.1499 - loss: 17.1901 - regularization_loss: 0.0000e+00 - total_loss: 17.1901
Epoch 3/10
86/86 [==============================] - 1s 10ms/step - root_mean_squared_error: 3.7224 - loss: 13.8229 - regularization_loss: 0.0000e+00 - total_loss: 13.8229
Epoch 4/10
86/86 [==============================] - 1s 9ms/step - root_mean_squared_error: 3.2325 - loss: 10.4189 - regularization_loss: 0.0000e+00 - total_loss: 10.4189
Epoch 5/10
86/86 [==============================] - 1s 9ms/step - root_mean_squared_error: 2.7290 - loss: 7.4236 - regularization_loss: 0.0000e+00 - total_loss: 7.4236
Epoch 6/10
86/86 [==============================] - 1s 10ms/step - root_mean_squared_error: 2.2558 - loss: 5.0716 - regularization_loss: 0.0000e+00 -

In [43]:
from tqdm import tqdm

def predict_all_user_food_pairs(model, uniqueUserIds, uniqueFoodIds, batch_size=64):
    predictions_df = pd.DataFrame(columns=["user_id", "recipe_id", "prediction"])
    
    i = 0
    for user_batch in tf.data.Dataset.from_tensor_slices(uniqueUserIds).batch(batch_size):
        i+=1
        print("i " , i)
        user_batch = tf.cast(user_batch, tf.string)
        user_embeddings = model.ranking_model.user_embeddings(user_batch)

        for food_batch in tf.data.Dataset.from_tensor_slices(uniqueFoodIds).batch(batch_size):
            food_batch = tf.cast(food_batch, tf.string)
            food_embeddings = model.ranking_model.product_embeddings(food_batch)

            user_embeddings_expanded = tf.repeat(user_embeddings, len(food_batch), axis=0)
            food_embeddings_expanded = tf.tile(food_embeddings, [len(user_batch), 1])

            batch_predictions = model.ranking_model.ratings(
                tf.concat([user_embeddings_expanded, food_embeddings_expanded], axis=1)
            )

            batch_df = pd.DataFrame({
                "user_id": tf.repeat(user_batch, len(food_batch)).numpy(),
                "recipe_id": tf.tile(food_batch, [len(user_batch)]).numpy(),
                "prediction": batch_predictions.numpy().flatten()
            })

            predictions_df = pd.concat([predictions_df, batch_df], ignore_index=True)

    return predictions_df

# Assuming you already have a trained model called 'model' and uniqueUserIds and uniqueFoodIds
print(uniqueFoodIds)
print([uniqueUserIds[0]])
# predictions_df = predict_all_user_food_pairs(model, [uniqueUserIds[0]], uniqueFoodIds)
# 
# average_predictions_per_user = predictions_df.groupby("user_id")["prediction"].mean().reset_index()
# 
# average_predictions_per_user['user_id'] = average_predictions_per_user['user_id'].str.decode('utf-8')



<StringArray>
[ '40893',  '44394',  '85009', '120345', '134728', '197160', '200236',
 '225241', '254596', '306785',
 ...
  '50804', '190698', '492861', '273409', '249924', '470995', '241491',
 '257796',  '72730', '386618']
Length: 231637, dtype: string
['38094']


In [29]:
predictions_df.to_csv('test.csv', index=False)

In [36]:
print(average_predictions_per_user)
print(predictions_df)

  user_id  prediction
0   38094    4.050269
         user_id  recipe_id  prediction
0       b'38094'   b'40893'    4.128861
1       b'38094'   b'44394'    4.034562
2       b'38094'   b'85009'    4.103045
3       b'38094'  b'120345'    4.100914
4       b'38094'  b'134728'    4.059107
...          ...        ...         ...
231632  b'38094'  b'470995'    4.102331
231633  b'38094'  b'241491'    3.998975
231634  b'38094'  b'257796'    4.097702
231635  b'38094'   b'72730'    4.092589
231636  b'38094'  b'386618'    4.026940

[231637 rows x 3 columns]


In [31]:
average_predictions_per_user.to_csv('average_predictions_per_user.csv', index=False)

In [42]:
# Save the weights using the `checkpoint_path` format
tf.saved_model.save(model, "models/")

INFO:tensorflow:Assets written to: models/assets


INFO:tensorflow:Assets written to: models/assets
